In [26]:
import mmengine
from pycocotools.coco import COCO
import json

import os
import torch
import torch.utils.data
import torchvision
import matplotlib.patches as patches
import matplotlib.pyplot as plt
from PIL import Image
import utils_ObjectDetection as utils
from tqdm import tqdm

In [27]:

# Reading annotations.json
TRAIN_ANNOTATIONS_PATH = "./train/_annotations.coco.json"
TRAIN_IMAGE_DIRECTIORY = "./train/"

VAL_ANNOTATIONS_PATH = "./valid/_annotations.coco.json"
VAL_IMAGE_DIRECTIORY = "./valid/"

train_coco = COCO(TRAIN_ANNOTATIONS_PATH)
valid_coco = COCO(VAL_ANNOTATIONS_PATH)

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [28]:
# Reading the annotation files
with open(TRAIN_ANNOTATIONS_PATH) as f:
  train_annotations_data = json.load(f)

with open(VAL_ANNOTATIONS_PATH) as f:
  val_annotations_data = json.load(f)
  
train_annotations_data['annotations'][500]


{'id': 500,
 'image_id': 33,
 'category_id': 2,
 'bbox': [338, 446, 69.16, 29.77],
 'area': 2058.893,
 'segmentation': [],
 'iscrowd': 0}

In [29]:
category_ids = train_coco.loadCats(train_coco.getCatIds())
print(category_ids)



[{'id': 1, 'name': 'cavity'}, {'id': 2, 'name': 'normal'}]


In [30]:
train_coco.getCatIds()


[1, 2]

In [31]:
im_id = train_coco.getImgIds(imgIds=335)
print(im_id)
img_info = train_coco.loadImgs(im_id)[0]
print(img_info)
# print(train_coco.getAnnIds(imgIds=img_info['id']))


[335]
{'id': 335, 'license': 1, 'file_name': 'healthy_teeth_394_jpg.rf.c4ba13d1347cac2b4de1c9e8eb20e0f6.jpg', 'height': 200, 'width': 252, 'date_captured': '2023-11-07T06:51:21+00:00'}


In [32]:
ann_ids = train_coco.getAnnIds(imgIds=img_info['id']) #img id, category id를 받아서 해당하는 annotation id 반환
anns = train_coco.loadAnns(ann_ids) # annotation id를 받아서 annotation 정보 반환
print(ann_ids)
print(anns)
 
# 처음에는 이해가 안갔는데 정리하면 한 이미지당 annotation된 충치가 여러개가 있다 
# 출력값을 보면 id가 0인 이미지의 annotation은 19개가 있는것이다.



[4018, 4019, 4020, 4021, 4022, 4023, 4024, 4025, 4026, 4027, 4028, 4029, 4030]
[{'id': 4018, 'image_id': 335, 'category_id': 2, 'bbox': [98, 117, 7.89, 10.73], 'area': 84.66, 'segmentation': [], 'iscrowd': 0}, {'id': 4019, 'image_id': 335, 'category_id': 2, 'bbox': [167, 118, 5.91, 9.27], 'area': 54.786, 'segmentation': [], 'iscrowd': 0}, {'id': 4020, 'image_id': 335, 'category_id': 2, 'bbox': [103, 115, 8.25, 15.37], 'area': 126.802, 'segmentation': [], 'iscrowd': 0}, {'id': 4021, 'image_id': 335, 'category_id': 2, 'bbox': [161, 117, 7.98, 13.54], 'area': 108.049, 'segmentation': [], 'iscrowd': 0}, {'id': 4022, 'image_id': 335, 'category_id': 2, 'bbox': [110, 116, 9.43, 17.44], 'area': 164.459, 'segmentation': [], 'iscrowd': 0}, {'id': 4023, 'image_id': 335, 'category_id': 2, 'bbox': [156, 119, 7.25, 14.76], 'area': 107.01, 'segmentation': [], 'iscrowd': 0}, {'id': 4024, 'image_id': 335, 'category_id': 2, 'bbox': [117, 117, 16.02, 20.98], 'area': 336.1, 'segmentation': [], 'iscrowd': 

In [33]:

import os
import torch
import torch.utils.data
import torchvision
from PIL import Image
from pycocotools.coco import COCO


class CavityDataset(torch.utils.data.Dataset):
    def __init__(self, root, annotation, transforms=None):
        self.root = root
        self.transforms = transforms
        self.coco = COCO(annotation)
        self.ids = list(sorted(self.coco.imgs.keys()))

    def __getitem__(self, index):
        # Own coco file
        coco = self.coco
        # Image ID
        img_id = self.ids[index]
        # List: get annotation id from coco
        ann_ids = coco.getAnnIds(imgIds=img_id)
        # Dictionary: target coco_annotation file for an image
        coco_annotation = coco.loadAnns(ann_ids)
        # path for input image
        path = coco.loadImgs(img_id)[0]['file_name']
        # open the input image
        img = Image.open(os.path.join(self.root, path))

        # number of objects in the image
        num_objs = len(coco_annotation)

        # Bounding boxes for objects
        # In coco format, bbox = [xmin, ymin, width, height]
        # In pytorch, the input should be [xmin, ymin, xmax, ymax]
        boxes = []
        for i in range(num_objs):
            xmin = coco_annotation[i]['bbox'][0]
            ymin = coco_annotation[i]['bbox'][1]
            xmax = xmin + coco_annotation[i]['bbox'][2]
            ymax = ymin + coco_annotation[i]['bbox'][3]
            boxes.append([xmin, ymin, xmax, ymax])
        boxes = torch.as_tensor(boxes, dtype=torch.float32)

        # labels = torch.ones((num_objs,), dtype=torch.int64)  # 충치있으면 1 없으면 2 ### 라벨 문제임
        labels = []
        for i in range(num_objs):
            labels.append(coco_annotation[i]['category_id'])
        labels = torch.as_tensor(labels, dtype=torch.int64)
        


        # Annotation is in dictionary format
        my_annotation = {}
        my_annotation["boxes"] = boxes
        my_annotation["labels"] = labels
 

        if self.transforms is not None:
            img = self.transforms(img)

        return img, my_annotation

    def __len__(self):
        return len(self.ids)

In [34]:
def get_transform():
    custom_transforms = []
    custom_transforms.append(torchvision.transforms.ToTensor())
    return torchvision.transforms.Compose(custom_transforms)

In [35]:
def train_set():
    train_data_dir = './train/'
    train_coco = './train/_train_anno.json'



    cavity_dataset = CavityDataset(root=train_data_dir,
                                annotation=train_coco,
                                transforms=get_transform())




    def collate_fn(batch): # 이거를 사용하면 resize안해도되네? x 상자모양 이상함
        return tuple(zip(*batch))


    train_batch_size = 8

    train_loader = torch.utils.data.DataLoader(dataset=cavity_dataset, 
                                            batch_size=train_batch_size,
                                            shuffle=True,
                                            num_workers=0,
                                            collate_fn=collate_fn
                                            )
    return cavity_dataset,train_loader
cavity_dataset,train_loader = train_set() 
print(cavity_dataset,train_loader)

loading annotations into memory...
Done (t=0.08s)
creating index...
index created!
<__main__.CavityDataset object at 0x0000022D989C2100> <torch.utils.data.dataloader.DataLoader object at 0x0000022D98CD95E0>


In [36]:
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# # DataLoader is iterable over Datasetgftrdec
# for imgs, annotations in train_loader:
#     imgs = list(img.to(device) for img in imgs)
#     annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]
#     print(annotations)


In [37]:
# train_coco = COCO(TRAIN_ANNOTATIONS_PATH)
# # anno_id = annotations[0]['image_id'].cpu().numpy()
# im_id = train_coco.getImgIds(imgIds=0)
# print(im_id)
# img_info = train_coco.loadImgs(im_id)[0]['file_name']
# print(img_info)

In [38]:
# from PIL import Image, ImageDraw
# ann = annotations[0]['boxes'].cpu().numpy()

# img = Image.open('./train/' + img_info).convert('RGB')

# draw = ImageDraw.Draw(img)
# for box in ann:
#     draw.rectangle((box[0], box[1], box[2], box[3]), outline=(0,255,0), width = 3)


# img.show()

In [39]:
# from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')


# def get_model_instance_segmentation(num_classes):
#     # load an instance segmentation model pre-trained pre-trained on COCO
#     model = torchvision.models.detection.fasterrcnn_resnet50_fpn(Weights=False)
#     # get number of input features for the classifier
#     in_features = model.roi_heads.box_predictor.cls_score.in_features
#     # replace the pre-trained head with a new one
#     model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

#     return model


# num_classes = 3  #### Faster R-CNN 사용 시 주의할 점은 background 클래스를 포함한 개수를 num_classes에 명시해야됨 ㅡㅡ 
# num_epochs = 10
# model = get_model_instance_segmentation(num_classes)

# # move model to the right device
# model.to(device)
# # parameters
# params = [p for p in model.parameters() if p.requires_grad]
# optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# len_dataloader = len(train_loader)


# for epoch in range(num_epochs):
#     model.train()
#     i = 0    
#     for imgs, annotations in train_loader:
#         i += 1
#         imgs = list(img.to(device) for img in imgs)

#         annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]

#         loss_dict = model(imgs, annotations)
#         losses = sum(loss for loss in loss_dict.values())

#         optimizer.zero_grad()
#         losses.backward()
#         optimizer.step()

#         print(f'Iteration: {i}/{len_dataloader}, Loss: {losses}')


        

In [40]:
# torch.save(model.state_dict(),f'model_{num_epochs}.pt')

# INFERENCE

In [41]:
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(Weights=False)
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model

num_classes = 3
num_epochs = 10
model = get_model_instance_segmentation(num_classes)

model.load_state_dict(torch.load(f'model_{num_epochs}.pt'))  

# move model to the right device
model.to(device)

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=1e-05)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=1e-05)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=1e-05)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=1e-05)
          (relu

In [42]:
# def plot_image_from_output(img, annotation):
    
#     img = img.cpu().permute(1,2,0)
    
#     fig,ax = plt.subplots(1)
#     ax.imshow(img)
    
#     for idx in range(len(annotation["boxes"])):
#         xmin, ymin, xmax, ymax = annotation["boxes"][idx]

#         if annotation['labels'][idx] == 1 :  # 충치에 빨간색 네모박스
#             rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=1,edgecolor='r',facecolor='none')
        
#         elif annotation['labels'][idx] == 2 :  # 정상이면 초록색 네모박스
            
#             rect = patches.Rectangle((xmin,ymin),(xmax-xmin),(ymax-ymin),linewidth=1,edgecolor='g',facecolor='none')
            

#         ax.add_patch(rect)

#     plt.show()

In [43]:
def test_set():
    test_data_dir = './valid/'
    test_coco = './valid/_annotations.coco.json'
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    def collate_fn(batch): 
        return tuple(zip(*batch))

    test_cavity_dataset = CavityDataset(root=test_data_dir,
                                        annotation=test_coco,
                                        transforms=get_transform())
    test_loader = torch.utils.data.DataLoader(dataset=test_cavity_dataset,
                                            batch_size=8,
                                            shuffle=True,
                                            collate_fn=collate_fn
                                            )
    return test_cavity_dataset, test_loader

test_cavity_dataset, test_loader=test_set()
print(test_cavity_dataset, test_loader)

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
<__main__.CavityDataset object at 0x0000022D98640970> <torch.utils.data.dataloader.DataLoader object at 0x0000022D8EB997F0>


In [44]:
# test_loader.__len__()
len_dataloader = len(test_loader)
print(len_dataloader)

11


In [45]:
def make_prediction(model, img, threshold):
    model.eval()
    preds = model(img)
    for id in range(len(preds)) :
        idx_list = []

        for idx, score in enumerate(preds[id]['scores']) :
            if score > threshold : 
                idx_list.append(idx)

        preds[id]['boxes'] = preds[id]['boxes'][idx_list]
        preds[id]['labels'] = preds[id]['labels'][idx_list]
        preds[id]['scores'] = preds[id]['scores'][idx_list]

    return preds

In [46]:
# with torch.no_grad():
#     for imgs, annotation in test_loader:
#         imgs = list(img.to(device) for img in imgs)

#         pred = make_prediction(model, imgs, 0.5)
#         pred = [{k: v.to(torch.device('cpu')) for k, v in t.items()} for t in pred] 
#         print(len(pred))


In [47]:
# _idx = 1
# print("Target : ", annotation[_idx]['labels'])
# plot_image_from_output(imgs[_idx], annotation[_idx])
# print("Prediction : ", pred[_idx]['labels'])
# plot_image_from_output(imgs[_idx], pred[_idx])

In [48]:
# from tqdm import tqdm

# labels = []
# preds_adj_all = []
# annot_all = []

# for im, annot in tqdm(test_loader, position = 0, leave = True):
#     im = list(img.to(device) for img in im)
#     #annot = [{k: v.to(device) for k, v in t.items()} for t in annot]

#     for t in annot:
#         labels += t['labels']

#     with torch.no_grad():
#         preds_adj = make_prediction(model, im, 0.5)
#         preds_adj = [{k: v.to(torch.device('cpu')) for k, v in t.items()} for t in preds_adj]
#         preds_adj_all.append(preds_adj)
#         annot_all.append(annot)  
#     print(len(preds_adj_all), len(annot_all))
#     print(preds_adj_all)

In [49]:
# import utils_ObjectDetection as utils
# sample_metrics = []
# for batch_i in range(len(preds_adj_all)):
#     sample_metrics += utils.get_batch_statistics(preds_adj_all[batch_i], annot_all[batch_i], iou_threshold=0.5) 

# print(sample_metrics)
# true_positives, pred_scores, pred_labels = [torch.cat(x, 0) for x in list(zip(*sample_metrics))]  # 배치가 전부 합쳐짐
# precision, recall, AP, f1, ap_class = utils.ap_per_class(true_positives, pred_scores, pred_labels, torch.tensor(labels))
# mAP = torch.mean(AP)
# print(f'mAP : {mAP}')
# print(f'AP : {AP}')


In [53]:
# 이거 돌려야됨

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
from torch.utils.data import Dataset, DataLoader

def get_model_instance_segmentation(num_classes):
    # load an instance segmentation model pre-trained pre-trained on COCO
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(Weights=True)
    # get number of input features for the classifier
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    return model


def main():
    num_classes = 3  #### Faster R-CNN 사용 시 주의할 점은 background 클래스를 포함한 개수를 num_classes에 명시해야됨 ㅡㅡ 
    num_epochs = 10
    model = get_model_instance_segmentation(num_classes)
    
    model.load_state_dict(torch.load(f'model_{num_epochs}.pt'))  
    # move model to the right device
    model.to(device)
    # parameters
    params = [p for p in model.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

    train_dataset, train_loader = train_set()
    test_dataset, test_loader = test_set()
    

    for epoch in range(num_epochs):
        print('[==={0} epoch===]'.format(epoch))
        # train_loss, train_mAP = train(model, train_loader, optimizer)
        mAP = test(model, test_loader,epoch)
    
    

def train(model, train_loader, optimizer):
    model.train()
    len_dataloader = train_loader.__len__()
    i = 0    
    for imgs, annotations in train_loader:
        i += 1
        imgs = list(img.to(device) for img in imgs)

        annotations = [{k: v.to(device) for k, v in t.items()} for t in annotations]

        loss_dict = model(imgs, annotations)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()
        #print(f'TRAIN Iteration: {i}/{len_dataloader}, Loss: {losses}, mAP: {mAP}')
        print(f'TRAIN Iteration: {i}/{len_dataloader}, Loss: {losses}')
    #return train_loss, train_mAP

# def mAP_result(model, epoch, loader):

       

#         for t in annot:
#             labels += t['labels']

#         with torch.no_grad():
#             preds_adj = make_prediction(model, im, 0.5)
#             preds_adj = [{k: v.to(torch.device('cpu')) for k, v in t.items()} for t in preds_adj]
#             preds_adj_all.append(preds_adj)
#             annot_all.append(annot)  

    
#     for batch_i in range(len(preds_adj_all)):
#         sample_metrics += utils.get_batch_statistics(preds_adj_all[batch_i], annot_all[batch_i], iou_threshold=0.5) 

#     true_positives, pred_scores, pred_labels = [torch.cat(x, 0) for x in list(zip(*sample_metrics))]  # 배치가 전부 합쳐짐
#     precision, recall, AP, f1, ap_class = utils.ap_per_class(true_positives, pred_scores, pred_labels, torch.tensor(labels))
#     mAP = torch.mean(AP)
#     print(f'mAP result Epoch: {epoch+1}, mAP: {mAP}')
#     return mAP




def test(model, test_loader, epoch):
    print("###########################################################")
    len_dataloader = test_loader.__len__()
    labels = []
    preds_adj_all = []
    annot_all = []
    sample_metrics = []
    total_losses = 0

    with torch.no_grad():
        i = 0 
        for imgs, annotations in test_loader:
            model.train()
            print(i)
            i += 1
            imgs = list(img.to(device) for img in imgs)
            annota = [{k: v.to(device) for k, v in t.items()} for t in annotations]
            loss_dict = model(imgs, annota)
            losses = sum(loss for loss in loss_dict.values())
            total_losses += losses
            
            for t in annotations:
                labels += t['labels']

            preds_adj = make_prediction(model, imgs, 0.5)
            preds_adj = [{k: v.to(torch.device('cpu')) for k, v in t.items()} for t in preds_adj]
            preds_adj_all.append(preds_adj)
            annot_all.append(annotations)  

        for batch_i in range(len(preds_adj_all)):
            sample_metrics += utils.get_batch_statistics(preds_adj_all[batch_i], annot_all[batch_i], iou_threshold=0.5) 

        true_positives, pred_scores, pred_labels = [torch.cat(x, 0) for x in list(zip(*sample_metrics))]  # 배치가 전부 합쳐짐
        precision, recall, AP, f1, ap_class = utils.ap_per_class(true_positives, pred_scores, pred_labels, torch.tensor(labels))
        mAP = torch.mean(AP)
        print(f'mAP result Epoch: {epoch+1}, mAP: {mAP}')
        return mAP
        
    
main()

loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
[===0 epoch===]
###########################################################
0
1
2
3
4
5
6
7
8
9
10
mAP result Epoch: 1, mAP: 0.706395335972964
[===1 epoch===]
###########################################################
0
1
2
3
4
5
6
7
8
9
10
mAP result Epoch: 2, mAP: 0.7072254025266562
[===2 epoch===]
###########################################################
0
1


KeyboardInterrupt: 